On va télécharger la page de l'édition courant de Sur les épaules de Darwin.

In [1]:
import requests
from bs4 import BeautifulSoup

In [42]:
r = requests.get('https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-18-mars-2017')

In [43]:
soup = BeautifulSoup(r.text, 'html.parser')

On va trouver l'élément `article`.

In [44]:
article = soup.find('article')

A partir de là, on peut trouver les titres : 

In [45]:
article.find_all('h2')

[<h2 class="chapo">
             Dans les traces du voyage...
         </h2>,
 <h2>Articles scientifiques :</h2>,
 <h2>Livres:</h2>,
 <h2>Liens internet :</h2>,
 <h2>La programmation musicale:</h2>]

In [46]:
elements = {}
for tag in article.find_all('h2'):
    elements[tag.text.strip()] = tag.next_sibling.text 

In [47]:
elements

{'Articles scientifiques :': 'Tobler R, Rohrlach A, Soubrier J, et coll. Aboriginal mitogenomes reveal 50,000 years of regionalism in Australia. Nature 2017, Mar 8 [publié en ligne, sous presse].Nielsen R, Akey JM, Jakobsson M, et coll. Tracing the peopling of the world through genomics. Nature 2017, 541:302-310.Nagle N, Ballantyne KN, van Oven M, et coll. Mitochondrial DNA diversity of present-day Aboriginal Australians and implications for human evolution in Oceania. Journal of Human Genetics 2017, 62:343-353.Mallick S, Li H, Lipson M, et coll. The Simons Genome Diversity Project: 300 genomes from 142 diverse populations. Nature 2016, 538:201-206.Pagani L, Lawson DJ, Jagoda E, et coll. Genomic analyses inform on migration events during the peopling of Eurasia. Nature 2016, 538:238-242.Malaspinas AS, Westaway MC, Muller C, et coll. A genomic history of Aboriginal Australia. Nature 2016, 538:207-214.Tucci S, Akey JM. Population genetics: A map of human wanderlust. Nature 2016, 538:179-

In [59]:
import re

In [69]:
article.find('h2', text=re.compile('Articles*')).next_sibling

<ul><li><p>Tobler R, Rohrlach A, Soubrier J, et coll. <strong>Aboriginal mitogenomes reveal 50,000 years of regionalism in Australia</strong>. Nature 2017, Mar 8 [publié en ligne, sous presse].</p></li><li><p>Nielsen R, Akey JM, Jakobsson M, et coll. <strong>Tracing the peopling of the world through genomics</strong>. Nature 2017, 541:302-310.</p></li><li><p>Nagle N, Ballantyne KN, van Oven M, et coll. <strong>Mitochondrial DNA diversity of present-day Aboriginal Australians and implications for human evolution in Oceania</strong>. Journal of Human Genetics 2017, 62:343-353.</p></li><li><p>Mallick S, Li H, Lipson M, et coll. <strong>The Simons Genome Diversity Project: 300 genomes from 142 diverse populations</strong>. Nature 2016, 538:201-206.</p></li><li><p>Pagani L, Lawson DJ, Jagoda E, et coll. <strong>Genomic analyses inform on migration events during the peopling of Eurasia</strong>. Nature 2016, 538:238-242.</p></li><li><p>Malaspinas AS, Westaway MC, Muller C, et coll. <strong>A

On peut écrire une fonction pour ça :

In [126]:
def get_content(soup):
    "Récupère le contenu de la page."
    elements = {}
    elements['titre'] = soup.find('h1', class_='cover-portrait-actions-title').text.strip()
    elements['article'] = soup.find('article')
    return elements

In [127]:
get_content(soup)

{'article': <article class="content-body"><p><strong>Bibliographie:</strong></p><p><strong>L'origine des espèces de Charles Darwin aux Editions Flammarion Origines,</strong></p><div class="onf-ad" id="sas_28990"></div><script type="text/javascript">
     window._sasFormats = window._sasFormats || [];
 
     window._sasFormats.push(function() {
         window.sas.call('std', {
             siteId:70662,
             pageId:537605,
             formatId:28990,
             target: ''
         });
     });
 </script><p><strong>Lettres choisies (1828-1859)de Charles Darwin traduit de l'anglais par Mickael Popelard préface de Stefen Jay Gould aux Editions Bayard</strong></p><p><strong>L'expression des émotions chez les hommes et les animaux de Charles Darwin aux Editions Rivages</strong></p><p><strong>La lettre volée, d'Edgar Allan Poe, Ed Folio Gallimard</strong></p><p><strong>La mal-mesure de l'homme de Stephen Jay Gould aux Editions Odile Jacob</strong></p><p><strong>Identité et violenc

On peut également trouver d'autres liens : 

In [128]:
def get_other_episodes(soup):
    "Trouve les liens vers d'autres épisodes de Darwin."
    base = 'https://www.franceinter.fr'
    links = soup.find_all('span', class_='preview-title')
    links = [base + link.a.attrs['href'] for link in links if link.a.attrs['href'].startswith('/emissions/sur-les-epaules-de-darwin/')]
    return links

In [129]:
soup.find_all('span', class_='preview-title')

[<span class="preview-title"><a href="/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-18-mars-2017">Arpenter le monde (2)</a></span>,
 <span class="preview-title"><a href="/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-mars-2017">Le temps du Rêve</a></span>,
 <span class="preview-title"><a href="/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-04-mars-2017">La caverne de Liang Bua</a></span>]

In [130]:
get_other_episodes(soup)

['https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-18-mars-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-mars-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-04-mars-2017']

On peut maintenant écrire un scraper.

In [131]:
content = {}
urls = ['https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-18-mars-2017']
while len(urls) > 0:
    url = urls.pop()
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    content[url] = get_content(soup)
    unseen_urls = [url for url in get_other_episodes(soup) if url not in content if url not in urls]
    urls.extend(unseen_urls)
    print('downloaded {}'.format(url.split('/')[-1]))

downloaded sur-les-epaules-de-darwin-18-mars-2017
downloaded sur-les-epaules-de-darwin-25-fevrier-2017
downloaded sur-les-epaules-de-darwin-04-mars-2017
downloaded sur-les-epaules-de-darwin-11-mars-2017


In [132]:
import pandas as pd

In [133]:
pd.DataFrame(content).transpose()

,article,titre
https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-04-mars-2017,"<article class=""content-body""><h2 class=""chapo...",La caverne de Liang Bua
https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-mars-2017,"<article class=""content-body""><h2 class=""chapo...",Le temps du Rêve
https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-18-mars-2017,"<article class=""content-body""><h2 class=""chapo...",Arpenter le monde (2)
https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-25-fevrier-2017,"<article class=""content-body""><h2 class=""chapo...",Arpenter le monde


# Et si on repartait de la page d'index des émissions 

In [86]:
r = requests.get('https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin')

In [88]:
soup = BeautifulSoup(r.text, 'html.parser')

In [99]:
['https://www.franceinter.fr' + link.attrs['href'] for link in soup.find_all('a', class_='rich-section-list-item-content-title')]

['https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-18-mars-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-mars-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-04-mars-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-25-fevrier-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-18-fevrier-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-fevrier-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-04-fevrier-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-28-janvier-2017',
 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-21-janvier-2017',
 'https://www.franceinter.fr

In [96]:
soup.find(class_='pager-item last').a.attrs['href']

'/emissions/sur-les-epaules-de-darwin?p=23'

In [137]:
urls = []
for page in range(1, 24):
    pageurl = 'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin?p={}'.format(page)
    r = requests.get(pageurl)
    soup = BeautifulSoup(r.text, 'html.parser')
    urls.extend(['https://www.franceinter.fr' + link.attrs['href'] for link in soup.find_all('a', class_='rich-section-list-item-content-title')])

In [138]:
len(urls)

341

Maintenant la boucle de scraping :

In [139]:
from ipywidgets import IntProgress
from IPython.display import display

In [140]:
progress = IntProgress(min=0, max=len(urls))
display(progress)
content = {}
for ind, url in enumerate(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    content[url] = get_content(soup)
    progress.value = ind

On peut construire le tableau de résultats.

In [226]:
df = pd.DataFrame(content).transpose()

In [227]:
df.describe()

,article,titre
count,341,341
unique,340,276
top,"<article class=""content-body""><blockquote clas...",La découverte du Nouveau Monde (3)
freq,2,3


Cherchons dans ces articles :

In [228]:
df['rawtext'] = [tag.text for tag in df.article]

On va aussi rajouter les dates dans l'index.

In [229]:
def convert(string):
    months = {'janvier':1,
     'fevrier':2,
     'mars':3,
     'avril':4,
     'mai':5,
     'juin':6,
     'juillet':7,
     'aout':8,
     'septembre':9,
     'octobre':10,
     'novembre':11,
     'decembre': 12}
    new_string = ' '.join(str(months.get(month) for month in string.split('-')[1]))
    return new_string

In [230]:
convert('01-aout-2015')

'< g e n e r a t o r   o b j e c t   c o n v e r t . < l o c a l s > . < g e n e x p r >   a t   0 x 1 2 b 9 e 8 0 4 8 >'

In [231]:
df.index[0]

'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-01-aout-2015'

In [241]:
results = df[df.rawtext.str.contains('antoine', case=False)]
results.shape

(2, 3)

In [242]:
from ipywidgets import interact
from IPython.display import HTML

In [243]:
@interact
def display_results(n=(0, results.index.size - 1)):
    display(HTML(data='<a href="{}">source</a>'.format(results.index[n]) + str(results.article.iloc[n])))

#  Sauvegarde

In [145]:
df.to_json('emissions.json')